# Analyzing AppInsights data

In this notebook, you will learn how to connect to your AppInsights data to understand what's going on with your application. The example below is the log data of a serverless app that I recently built using Azure Function, which serves as an HTTP API to execute a PowerShell notebook.

## Prerequisites

1. [Kqlmagic](https://docs.microsoft.com/sql/azure-data-studio/notebooks/notebooks-kqlmagic) is installed.
2. [Azure Application Insights](https://docs.microsoft.com/en-us/azure/azure-monitor/app/app-insights-overview) resource is up and running.  
This notebook is based on log data collected for Azure Functions in App Insights - see [supported features](https://docs.microsoft.com/en-us/azure/azure-monitor/app/azure-functions-supported-features) for more info. Note that [Azure Functions](https://docs.microsoft.com/en-us/azure/azure-functions/functions-overview) offers built-in integration with Azure Application Insights to monitor functions.

# 1. Load Kqlmagic

Hide banner

In [1]:
%env KQLMAGIC_LOAD_MODE=silent

env: KQLMAGIC_LOAD_MODE=silent


Load Kqlmagic

> **Note**: I'm using Kqlmagic 0.1.114post2. 

In [2]:
%reload_ext Kqlmagic

# 2. Get AppId of the Application Insights

To obtain this, 
1. Login to [Azure Portal](https://portal.azure.com)
2. Navigate to the Application Insights resource that you want to connect to.
3. From the left menu, choose API Access. You should see Application ID (guid format). 

Once you obtain the Application ID, simply copy and paste the value to the below connection string, i.e. after `appid=`. Make sure you surround the guid with `'` (single quote).

![How to get Azure Application Insights app id](https://www.mssqlgirl.com/wp-content/uploads/2021/08/image.png)

_Source MsSQLGirl.com blog post on [Analyzing your Azure serverless app performance stored in Application Insights using Kqlmagic](https://www.mssqlgirl.com/analyzing-your-azure-serverless-app-performance-stored-in-application-insights-using-kqlmagic/)_

In [ ]:
%kql appinsights://code;appid='aefefbbc-99b7-88ce-0023-2339238abc392b8';alias='myServerlessAppLog' -try-azcli-login-by-profile -!se

> **Tip**:   
> - use `-!se` in case there is any error. Very useful for analysis.  
> - use `-try-azcli-login-by-profile` for SSO authentication. 

## 3. Response time buckets 

Show how many requests are in each performance-bucket. 

In [5]:
%%kql requests
| summarize requestCount=sum(itemCount), avgDuration=avg(duration) by performanceBucket
| order by avgDuration asc // sort by average request duration
| project-away avgDuration // no need to display avgDuration, we used it only for sorting results
| render barchart

[{'performanceBucket': '<250ms', 'requestCount': 15},
 {'performanceBucket': '250ms-500ms', 'requestCount': 13},
 {'performanceBucket': '500ms-1sec', 'requestCount': 1},
 {'performanceBucket': '3sec-7sec', 'requestCount': 1},
 {'performanceBucket': '7sec-15sec', 'requestCount': 8},
 {'performanceBucket': '1min-2min', 'requestCount': 1}]

## 4. Response time trend

Chart request duration over the last 12 hours. 

In [6]:
%%kql requests
| where timestamp > ago(12h) 
| summarize avgRequestDuration=avg(duration) by bin(timestamp, 10m) // use a time grain of 10 minutes
| render timechart

[{'timestamp': datetime.datetime(2021, 8, 1, 21, 30, tzinfo=tzutc()), 'avgRequestDuration': 8639.4051},
 {'timestamp': datetime.datetime(2021, 8, 1, 21, 40, tzinfo=tzutc()), 'avgRequestDuration': 391.6498},
 {'timestamp': datetime.datetime(2021, 8, 1, 16, 50, tzinfo=tzutc()), 'avgRequestDuration': 11050.9801}]

## 5. Operations performance 
Calculate request count and duration by operations. 

In [7]:
%%kql requests
| summarize RequestsCount=sum(itemCount), AverageDuration=avg(duration), percentiles(duration, 50, 95, 99) by operation_Name // you can replace 'operation_Name' with another value to segment by a different property
| order by RequestsCount desc // order from highest to lower (descending)


,operation_Name,RequestsCount,AverageDuration,percentile_duration_50,percentile_duration_95,percentile_duration_99
0,HttpExamplePosh,39,4665.07671,288.5339,12515.4669,94545.9267


[{'operation_Name': 'HttpExamplePosh', 'RequestsCount': 39, 'AverageDuration': 4665.07671025641, 'percentile_duration_50': 288.5339, 'percentile_duration_95': 12515.4669, 'percentile_duration_99': 94545.9267}]

## 6. Request Count Trend

Chart Request count over the last day. 


In [8]:
%%kql requests
| summarize totalCount=sum(itemCount) by bin(timestamp, 30m)
| render timechart

[{'timestamp': datetime.datetime(2021, 7, 31, 12, 0, tzinfo=tzutc()), 'totalCount': 2},
 {'timestamp': datetime.datetime(2021, 8, 1, 12, 30, tzinfo=tzutc()), 'totalCount': 4},
 {'timestamp': datetime.datetime(2021, 8, 1, 13, 0, tzinfo=tzutc()), 'totalCount': 5},
 {'timestamp': datetime.datetime(2021, 8, 1, 16, 30, tzinfo=tzutc()), 'totalCount': 1},
 {'timestamp': datetime.datetime(2021, 7, 31, 4, 30, tzinfo=tzutc()), 'totalCount': 11},
 {'timestamp': datetime.datetime(2021, 8, 1, 21, 30, tzinfo=tzutc()), 'totalCount': 2},
 {'timestamp': datetime.datetime(2021, 8, 1, 5, 0, tzinfo=tzutc()), 'totalCount': 10},
 {'timestamp': datetime.datetime(2021, 8, 1, 4, 0, tzinfo=tzutc()), 'totalCount': 1},
 {'timestamp': datetime.datetime(2021, 8, 1, 4, 30, tzinfo=tzutc()), 'totalCount': 1},
 {'timestamp': datetime.datetime(2021, 8, 1, 6, 30, tzinfo=tzutc()), 'totalCount': 2}]

## 7.  Top 10 countries by traffic 

Chart the amount of requests from the top 10 countries. 

In [9]:
%%kql requests
| summarize CountByCountry=count() by client_CountryOrRegion
| top 10 by CountByCountry
| render piechart

[{'client_CountryOrRegion': 'United States', 'CountByCountry': 37},
 {'client_CountryOrRegion': 'Australia', 'CountByCountry': 2}]